# Brown-Clustering based Rewriting
Goal: Re-write a dataset **without external data or models**; improve upon crude strategy provided below


##  Do Brown-Clustering

In [1]:
from datasets import load_dataset
from transformers import RobertaTokenizerFast

/data/loriss21dm/TracInVenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
args = {
    "dataset_folder": "./train_10M",
    "model_dir": "./10MModel",
}


dataset = load_dataset("text", data_dir=args["dataset_folder"])
tokenizer = RobertaTokenizerFast.from_pretrained(args["model_dir"], max_len=512)
sentences = [tokenizer.tokenize(t) for t in dataset["train"]["text"]]
# ignore maximum sequence lenght warning, we want all tokens when clustering

/data/loriss21dm/TracInVenv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (614 > 512). Running this sequence through the model will result in indexing errors


In [3]:
from brown_clustering import BigramCorpus, BrownClustering
# corpus = BigramCorpus(sentences, alpha=0.5, min_count=0)
# corpus.print_stats()



In [4]:
import cloudpickle

In [5]:
# clustering = BrownClustering(corpus, m=1000)

# clusters = clustering.train()

# with open('brown_clustering', 'wb') as handle:
#     cloudpickle.dump(clustering, handle)

## Re-write data

In [6]:
clustering = None
with open('brown_clustering', "rb") as handle:
    clustering = cloudpickle.load(handle)

`clustering` provides a binary tree, the longer the path, the more similar sibling tokens are. `clustering.get_similar` 

In [7]:
clustering.get_similar('Ġme')

[('Ġhim', '0010101110100'),
 ('Ġthem', '0010101110101'),
 ('Ġus', '001010111011'),
 ('Ġoff', '00101011111010'),
 ('Ġaway', '00101011111011'),
 ('Ġdown', '00101011111000'),
 ('Ġover', '00101011111001'),
 ('Ġup', '001010111100'),
 ('Ġout', '001010111101'),
 ('Ġback', '001010111111')]

In [8]:
# DO NOT print(clustering.codes())

### Just replace one token
With a random sibling, anywhere uptree/sibling

In [9]:

# IDX = 3
# import random
# for _ in range(0,10):
#     result = []
#     for i, word in enumerate(sentences[59]):
#         replacement = clustering.get_similar(word)
        
#         if i == IDX and len(replacement):
#             result.append(random.choice(replacement)[0])
#         else:
#             result.append(word)
    
#     print(tokenizer.convert_tokens_to_string(result))

##  Crude re-writing strategy
Replace between 1 and 10% of the tokens in the document with the next most similar word/sibling, but only if their path lenght in the binary tree is at least `L` (max would be num clusters, this is the only measure of similarity available here)

In [10]:
# from random import sample
# random.seed(42)
# L = 500
# max_pct_to_replace = 10
# import random


# for sentence in sample(sentences[0:10000], 2000):
#     result = []
    
#     replaced = 0
#     for i, word in enumerate(sentence):
#         #  i > 0 : ignore sentence start for now, bad performance there artifact of tokenizer
#         if i > 0 and word in clustering.codes() and len(clustering.codes()[word]) >= L and (replaced/len(sentence) < max_pct_to_replace or replaced == 0):
#             result.append(clustering.get_similar(word)[0][0]) # just get next most similar word/sibling in tree
#             replaced +=1
#         else:
#             result.append(word)
#     if replaced > 0:
#         print(tokenizer.convert_tokens_to_string(sentence),"---->",tokenizer.convert_tokens_to_string(result))

### Replace N words with longest paths
Replace up to `max_ratio_to_replace` of tokens with siblings. Chooses tokens with longer paths. 

TODO: Super inefficent, leverage tree structure properly instead of doing "len()"


In [11]:
import heapq
import math


In [26]:
# convert to dicts of string:string from string:heapq for efficency
codes = dict(clustering.codes())
inverse_dict = {v: k for k, v in codes.items()}
path_lenghts = {k:len(v) for k,v in codes.items()}

In [27]:
word = "ĠNovember"
code = int(codes[word], base=2)
print("{0:b}".format(code))
print(inverse_dict["{0:b}".format(code)])
print("{0:b}".format(code ^ 1)) # get closest sibling
print(inverse_dict["{0:b}".format(code ^ 1)])

1010101011011110000
ĠNovember
1010101011011110001
ĠApril


In [25]:
clustering.get_similar("ĠNovember")

[('ĠApril', '1010101011011110001'),
 ('ĠOctober', '101010101101111001'),
 ('ĠMay', '10101010110111101'),
 ('ĠSeptember', '1010101011011111'),
 ('ĠJanuary', '1010101011011100'),
 ('ĠJuly', '1010101011011101'),
 ('ĠMarch', '10101010110110'),
 ('ĠAugust', '1010101011010'),
 ('ĠJune', '101010101100'),
 ('ĠDecember', '10101010111')]

In [30]:
from random import sample
import random
random.seed(0)

max_ratio_to_replace = 0.1


def save_replacement(word):
    """
    Some words have no siblings, this is faster than checking if this is the case
    """
    try: 
        return inverse_dict["{0:b}".format(int(codes[word], base=2) ^ 1)]
    except:
        return word


for sentence in sample(sentences[0:10000], 2000):
    lens = [path_lenghts[word] for word in sentence] # get path lenght for each word in this sentence
    largest_paths = [lens.index(i) for i in heapq.nlargest(int(math.ceil(len(lens) * max_ratio_to_replace)), lens)] # get max_ratio_to_replace longest paths

    result = [ save_replacement(word) if (i in largest_paths) else word 
    for i, word in enumerate(sentence) ]
    
    if result != sentence: # TODO defaults to returning the original string if no replacement was possible for now
        print(tokenizer.convert_tokens_to_string(sentence),"---->",tokenizer.convert_tokens_to_string(result))


Ah, the first Saturday in every month. ----> Please, the first Saturday in every month.
Ah, that's right, sorry  ----> Please, that's right, sorry 
That's interesting. ----> That's important.
Yeah , you could say that. ----> Yeah   you could say that.
I would have them like  shoes. ----> I would have them like  clothes.
I like that! ----> I 1936 that!
Not  are we? ----> Just  are we?
And we send them the replacement, and they let the broken one sit there. ----> And we kill them the replacement, and they let the broken one sit there.
Ah no, racing, ah, it's cos I bought a horse, that's why ----> Please no, racing, ah, it's cos I bought a horse, that's why
Er, about six years. ----> Erm, about six years.
On erm food and noise, we're still very, very busy indeed, and our figure for noise inspection is higher than it ever has been before, and the comment that was made under that section will show you that some of that most certainly is the amount of work that the  team had to carry out dur

## TODO

- Find good hyperparameters
- Improve re-writing strategy
- Evaluate quality of re-writes somehow
- Probably evaluate cluster quality somehow



### Tree Structure
Not used yet, could potentialy choose words uptree and not just sibling

In [11]:
codes = [int(code, base=2) for code in clustering.codes().values()]
words = list(clustering.codes().keys())

In [ ]:
clustering.codes().values()

In [75]:
for code in codes:
    print("{0:b}".format(code))
    print("{0:b}".format(code ^ 1)) # get closest sibling
    print("-------")

1100000
1100001
-------
1100001
1100000
-------
110001
110000
-------
11001
11000
-------
1101
1100
-------
111
110
-------
10
11
-------
0
1
-------
1
0
-------


{'0000': 'This',
 '0001': 'an',
 '001': 'another',
 '01': 'is',
 '10': 'example',
 '11': 'sentence'}